# Cleaning and Transforming Data for F1 Machine Learning Model 

### Imports

In [3]:
import fastf1
import pandas as pd
fastf1.Cache.enable_cache("cache")
import matplotlib.pyplot as plt 



## References
- https://medium.com/@nityachintan/how-i-built-an-f1-race-prediction-app-as-my-first-machine-learning-project-7e2e9cc89826
- https://docs.fastf1.dev/
- https://arno.uvt.nl/show.cgi?fid=180319


## Cleaning Race Data

In [4]:
race_2024_laps = pd.read_csv("Raw_Data/2024_Race_Laps.csv")
race_2024_results = pd.read_csv("Raw_Data/2024_Race_Results.csv")
race_2024_weather = pd.read_csv("Raw_Data/2024_Race_Weather.csv")

race_2025_laps = pd.read_csv("Raw_Data/2025_Race_Laps.csv")
race_2025_results = pd.read_csv("Raw_Data/2025_Race_Results.csv")
race_2025_weather = pd.read_csv("Raw_Data/2025_Race_Weather.csv")

### Creating Year Column

In [5]:
race_2024_laps["Year"] = 2024

In [6]:
race_2025_laps["Year"] = 2025

### Creating DriverId Column

In [7]:
# Creating a map to map the Drivers abreviation to an ID
driver_id_map = {
    'VER':'max_verstappen',
    'GAS':'gasly',
    'ANT':'antonelli',
    'ALO':'alonso',
    'LEC':"leclerc",
    'STR':'stroll',
    'TSU':'tsunoda',
    'ALB':'albon',
    'HUL':'hulkenberg',
    'LAW':'lawson', 
    'OCO':'ocon',
    'NOR':'norris', 
    'HAM':'hamilton',
    'BOR':'bortoleto', 
    'SAI':'sainz',
    'HAD':"hadjar",
    'RUS':'russell',
    'PIA':"piastri",
    'BEA':"bearman",
    'COL':"colapinto",
    'DOO':"doohan",
    "PER":"perez",
    "ZHO":"zhou",
    "MAG":"kevin_magnussen",
    "RIC":"ricciardo",
    "BOT":"bottas",
    "SAR":"sargeant"

}

# Mapping by Taking the driver column values and locating it within the driver id map
race_2024_laps["DriverID"] = race_2024_laps["Driver"].map(driver_id_map).str.strip().astype("string")

race_2025_laps["DriverID"] = race_2025_laps["Driver"].map(driver_id_map).str.strip().astype("string")

### Dropping Redundant/Null Columns

In [8]:
cols_to_drop_laps = ["DeletedReason","IsAccurate","FastF1Generated","IsPersonalBest","LapStartDate"]

In [9]:
race_2024_laps.drop(columns=cols_to_drop_laps,inplace=True)

race_2025_laps.drop(columns=cols_to_drop_laps,inplace=True)

### Time Data Columns

In [10]:
time_columns = ['Time','LapTime','PitOutTime', 'PitInTime', 'Sector1Time', 'Sector2Time', 'Sector3Time','Sector1SessionTime', 'Sector2SessionTime', 'Sector3SessionTime',"LapStartTime"]

race_2025_laps[time_columns] = race_2025_laps[time_columns].apply(pd.to_timedelta)

race_2024_laps[time_columns] = race_2024_laps[time_columns].apply(pd.to_timedelta)


## Race Weather Data

### Creating Year Column

In [11]:
race_2024_weather["Year"] = 2024

race_2025_weather["Year"] = 2025

### Weather Time Columns

In [12]:
race_2024_weather["Time"] = pd.to_timedelta(race_2024_weather["Time"])

race_2025_weather["Time"] = pd.to_timedelta(race_2025_weather["Time"])

### Merging Race Weather Data with Race Lap Data

In [13]:
# Creating empty list to append data too
merged_data_2024 = []

# Looping through each roundnumber in the 2024 dataset and sorting values to ensure it loops in the correct order
for round_2024 in sorted(race_2024_laps["RoundNumber"].unique()):

    # Creating a variables for the lap data and sorting values for later merge
    laps_2024 = (
        race_2024_laps[race_2024_laps["RoundNumber"] == round_2024].sort_values("Time")
    )

    # Creating weather data variable where the round number is equal to the part of the loop
    weather_2024 = (
        race_2024_weather[race_2024_weather["RoundNumber"] == round_2024].sort_values("Time")
    )

    # Merging the two variables on time using direction backword to ensure it takes the weather data less than or eqaul to that time period
    merge_2024 = pd.merge_asof(
        laps_2024, weather_2024, on="Time", direction="backward"
    )

    # Finally appending the data to the original empty list
    merged_data_2024.append(merge_2024)

# Concatinating the data into one dataset 
race_2024_data = pd.concat(merged_data_2024, ignore_index=True)



In [14]:
# Dropping and renaming columns that were created during the merge
cols_drop = ['RoundNumber_y', 'Year_y']

cols_rename = {'RoundNumber_x':'RoundNumber', 'Year_x':'Year'}

race_2024_data.drop(columns=cols_drop,inplace=True)

race_2024_data.rename(columns=cols_rename,inplace=True)

In [15]:
# Same process as above but for 2025 data
merged_data_2025 = []

for round_2025 in sorted(race_2025_laps["RoundNumber"].unique()):
    laps_2025 = (
        race_2025_laps[race_2025_laps["RoundNumber"] == round_2025].sort_values("Time")
    )

    weather_2025 = (
        race_2025_weather[race_2025_weather["RoundNumber"] == round_2025].sort_values("Time")
    )

    merge_2025 = pd.merge_asof(
        laps_2025, weather_2025, on="Time", direction="backward"
    )

    merged_data_2025.append(merge_2025)


race_2025_data = pd.concat(merged_data_2025, ignore_index=True)

In [16]:
cols_drop = ['RoundNumber_y', 'Year_y']

cols_rename = {'RoundNumber_x':'RoundNumber', 'Year_x':'Year'}

race_2025_data.drop(columns=cols_drop,inplace=True)

race_2025_data.rename(columns=cols_rename,inplace=True)

### Joining 2025 and 2024 Datasets

In [17]:
merger = [race_2024_data, race_2025_data]

race_data = pd.concat(merger, ignore_index=True)

### Writing Data to CSV

In [18]:
race_data.to_csv("Cleaned_Data/Cleaned_Race_Data.csv",index=False)

## Inspection of Cleaned Race Data

In [19]:
full_df = pd.read_csv("Cleaned_Data/Cleaned_Race_Data.csv")

In [20]:
full_df.shape

(48938, 36)

## Null Values

In [21]:
print(f"Percentage of null values : {(full_df.isnull().sum() / len(full_df)) * 100}")

Percentage of null values : Time                   0.000000
Driver                 0.000000
DriverNumber           0.000000
LapTime                1.567289
LapNumber              0.000000
Stint                  0.911357
PitOutTime            96.789816
PitInTime             96.820467
Sector1Time            1.994360
Sector2Time            0.165516
Sector3Time            0.214557
Sector1SessionTime     2.210961
Sector2SessionTime     0.165516
Sector3SessionTime     0.214557
SpeedI1               15.721934
SpeedI2                0.210470
SpeedFL                3.340962
SpeedST                8.077568
Compound               1.027831
TyreLife               1.074829
FreshTyre              0.000000
Team                   0.000000
LapStartTime           0.000000
TrackStatus            0.000000
Position               0.116474
Deleted                0.000000
RoundNumber            0.000000
Year                   0.000000
DriverID               0.000000
AirTemp                0.000000
Humidity    

## Driver Column

In [22]:
# Driver and Driver ID match so drop Driver Column
full_df[["Driver","DriverID"]].value_counts()

full_df.drop(columns="Driver",inplace=True)

## Driver Number Column

In [23]:
# Similar to Driver Column not needed due to Driver ID Column
full_df[["DriverID", "DriverNumber"]].value_counts()

full_df.drop(columns="DriverNumber", inplace=True)

## LapTime Column

In [24]:
# Convert to timedlta for time series aggregation
full_df["LapTime"] = pd.to_timedelta(full_df["LapTime"])

In [25]:
# Convert to total seconds for easier analysis
full_df["LapTime"] = full_df["LapTime"].dt.total_seconds()

In [26]:
# 548 Null values will need to fill these values 
full_df["LapTime"].isnull().sum()

767

## LapNumber Column

In [27]:
# Dropping First lap data as will skew results
full_df = full_df[full_df["LapNumber"] != 1]

## Stint Column

In [28]:
# 427 Null values here will need to fill these seems to be the first 24 laps from round number 6 in year 2025
full_df["Stint"].isnull().sum()
null_stint_values = full_df[full_df["Stint"].isnull()]

## Pit In and out Time Column

In [29]:
# Mostly null values wont be used in final dataset
# Will use these columns to create a laptype column that specifies whether the lap is an inlap, outlap or normal lap
full_df["PitInTime"].isnull().sum()

46558

In [30]:
full_df["PitOutTime"].isnull().sum()

46545

## Sector 1, 2, 3 Time Columns

In [31]:
sector_times = ['Sector1Time', 'Sector2Time', 'Sector3Time']

# Very small amount of null values may just drop these rows 
full_df[sector_times].isnull().sum()

full_df[sector_times] = full_df[sector_times].apply(pd.to_timedelta)

In [32]:
# Deleting data where the sector times are null
full_df = full_df[~full_df["Sector1Time"].isnull()]
full_df = full_df[~full_df["Sector2Time"].isnull()]
full_df = full_df[~full_df["Sector3Time"].isnull()]

In [33]:
# Converting to Total Seconds similar to laptimes 
full_df["Sector1Time"] = full_df["Sector1Time"].dt.total_seconds()
full_df["Sector2Time"] = full_df["Sector2Time"].dt.total_seconds()
full_df["Sector3Time"] = full_df["Sector3Time"].dt.total_seconds()


## Sector 1, 2, 3 Session Times

In [34]:
# Not neccesary for analysis so will be dropped
ses_times = ['Sector1SessionTime',
       'Sector2SessionTime', 'Sector3SessionTime']
full_df.drop(columns=ses_times, inplace=True)

## Speed Trap Columns

In [35]:
st = ['SpeedI1', 'SpeedI2',
       'SpeedFL', 'SpeedST']

# Some null values, dont think will use these columns but will be evaluated further for now
full_df[st].isnull().sum()

SpeedI1    7613
SpeedI2      24
SpeedFL    1495
SpeedST    3852
dtype: int64

## Compound Column

In [36]:
full_df["Compound"].isnull().sum()
# 484 values are null will have to try fill these

484

## TyreLife Column

In [37]:
full_df[full_df["TyreLife"].isnull()]
# 507 null values will need to fill these

,Time,LapTime,LapNumber,Stint,PitOutTime,PitInTime,Sector1Time,Sector2Time,Sector3Time,SpeedI1,...,RoundNumber,Year,DriverID,AirTemp,Humidity,Pressure,Rainfall,TrackTemp,WindDirection,WindSpeed
27281,0 days 02:24:57.034000,113.822,40.0,5.0,0 days 02:23:05.453000,NaN,47.593,25.942,40.287,146.0,...,1,2025,bearman,16.4,69.0,1010.3,False,19.3,255,3.6
27297,0 days 02:26:26.183000,89.149,41.0,5.0,NaN,NaN,31.306,19.948,37.895,252.0,...,1,2025,bearman,16.5,69.0,1010.4,False,19.3,262,3.9
27313,0 days 02:27:53.786000,87.603,42.0,5.0,NaN,NaN,30.422,19.362,37.819,257.0,...,1,2025,bearman,16.6,68.0,1010.6,False,19.3,222,5.1
27329,0 days 02:29:24.843000,91.057,43.0,5.0,NaN,NaN,30.290,19.299,41.468,263.0,...,1,2025,bearman,16.6,75.0,1010.5,False,19.3,201,1.3
27351,0 days 02:31:27.804000,122.961,44.0,5.0,NaN,0 days 02:31:10.506000,30.419,19.282,73.260,268.0,...,1,2025,bearman,16.1,77.0,1010.3,True,19.3,0,3.5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
40696,0 days 03:41:08.526000,106.061,44.0,3.0,NaN,NaN,30.499,46.821,28.741,333.0,...,13,2025,hulkenberg,16.9,83.0,965.2,False,22.4,220,1.1
40700,0 days 03:41:21.679000,106.911,44.0,3.0,NaN,NaN,30.734,47.506,28.671,340.0,...,13,2025,antonelli,16.9,83.0,965.2,False,22.4,220,1.1
40701,0 days 03:41:22.877000,107.306,44.0,3.0,NaN,NaN,31.011,46.969,29.326,333.0,...,13,2025,alonso,16.9,83.0,965.2,False,22.4,220,1.1
40702,0 days 03:41:26.977000,106.423,44.0,3.0,NaN,NaN,31.206,46.291,28.926,319.0,...,13,2025,sainz,16.9,83.0,965.2,False,22.4,220,1.1


## FreshTyre

In [38]:
# No Null values not sure whether this will be useful for analysis but will be taken for now
full_df["FreshTyre"].value_counts()

FreshTyre
True     37869
False    10081
Name: count, dtype: int64

## Team Column

In [39]:
# No Null values and data is correct may create a power ranking type column evaluating team performance on that round number
full_df[["DriverID","Team"]].value_counts()

DriverID         Team           
norris           McLaren            2585
russell          Mercedes           2563
piastri          McLaren            2554
leclerc          Ferrari            2554
max_verstappen   Red Bull Racing    2481
alonso           Aston Martin       2353
stroll           Aston Martin       2351
gasly            Alpine             2296
albon            Williams           2183
bottas           Kick Sauber        1333
hamilton         Mercedes           1320
zhou             Kick Sauber        1318
sainz            Ferrari            1316
hulkenberg       Haas F1 Team       1276
bearman          Haas F1 Team       1249
perez            Red Bull Racing    1239
tsunoda          RB                 1223
ocon             Alpine             1213
kevin_magnussen  Haas F1 Team       1211
ocon             Haas F1 Team       1146
hamilton         Ferrari            1137
bortoleto        Kick Sauber        1092
hadjar           Racing Bulls       1090
hulkenberg       Kick Sa

## LapStartTime Column

In [40]:
# No null values, unsure whether this will be useful or not so will be kept for now 
full_df["LapStartTime"].head()

20    0 days 01:01:37.489000
21    0 days 01:01:38.476000
22    0 days 01:01:39.433000
23    0 days 01:01:40.258000
24    0 days 01:01:40.825000
Name: LapStartTime, dtype: object

## TrackStatus

In [41]:
# No Null Values, will be important as this shows if there were any yellow flags, safety car, etc.
full_df["TrackStatus"].isnull().sum()

0

In [42]:
ts_mapping = {
    1 : "Track Clear",
    2 : "Yellow Flag",
    4 : "Safety Car",
    5 : "Red Flag",
    6 : "VSC Deployed",
    7 : "VSC Ending"
}

full_df["TrackStatus"] = full_df["TrackStatus"].map(ts_mapping).str.strip().astype("string")

In [43]:
# Will help in identifying any irregular lap times, will keep for now 
full_df["TrackStatus"] = full_df["TrackStatus"].fillna("Unknown")

## Position Column

In [44]:
# No Null values and all values seem correct, will be keeping this column
full_df["Position"].isnull().sum()

0

## Deleted Column

In [45]:
# No null values, with values either being true or false may help in identifying laps that may skew the data will keep for now
full_df["Deleted"].isnull().sum()

0

## AirTemp Column

In [46]:
# No Null values, will keep for now may affect lap times 
full_df["AirTemp"].isnull().sum()

0

## Humidity Column

In [47]:
# No Null values, will keep for now 
full_df["Humidity"].isnull().sum()

0

## Pressure

In [48]:
# No Null values, will keep for now
full_df["Pressure"].isnull().sum()

0

## Rainfall Column

In [49]:
# No Null values, either true or false, will definitley affect lap times
full_df["Rainfall"].value_counts()

Rainfall
False    46018
True      1932
Name: count, dtype: int64

## TrackTemp

In [50]:
# May effect lap times, will keep for now, no null values
full_df["TrackTemp"].isnull().sum()

0

## WindDirection

In [51]:
# No Null values, will keep for now may effect lap times
full_df["WindDirection"].isnull().sum()

0

## WindSpeed Column

In [52]:
# No Null values, will be taken for now to see if it effects lap times 
full_df["WindSpeed"].isnull().sum()

0

## Time Column

In [53]:
# Dropping as unnesecary for analysis
full_df.drop(columns="Time",inplace=True)

## Final Dataset after Inspection

In [107]:
full_df.isnull().sum()

LapTime           460
LapNumber           0
Stint             427
Sector1Time         0
Sector2Time         0
Sector3Time         0
SpeedI1          7613
SpeedI2            24
SpeedFL          1495
SpeedST          3852
Compound          484
TyreLife          507
FreshTyre           0
Team                0
LapStartTime        0
TrackStatus         0
Position            0
Deleted             0
DriverID            0
AirTemp             0
Humidity            0
Pressure            0
Rainfall            0
TrackTemp           0
WindDirection       0
WindSpeed           0
LapType             0
RoundYear           0
MedianLapTime       0
Rank                0
dtype: int64

# Feauture Creation 

## Lap Type

In [66]:
def get_lap_type(row):
    if pd.notnull(row["PitInTime"]):
        return "InLap"
    elif pd.notnull(row["PitOutTime"]):
        return "OutLap"
    else:
        return "Lap"
    
full_df["LapType"] = full_df.apply(get_lap_type,axis=1)

full_df.drop(columns={"PitInTime", "PitOutTime"}, inplace=True)

## New Round Number Column

In [68]:
full_df["Year"] = full_df["Year"].astype("string")

full_df["RoundNumber"] = full_df["RoundNumber"].astype("string")

def get_round_col(row):
    if row["Year"] == "2024":
        return "2024_" + row["RoundNumber"]
    else:
        return "2025_" + row["RoundNumber"]
    
full_df["RoundYear"] = full_df.apply(get_round_col,axis=1)

full_df.drop(columns={"Year", "RoundNumber"},inplace=True)

## Team Power Ranking

In [ ]:
# Creating Power Ranking by taking each teams median lap time
# Firstly grouping by round number and Team, taking the median laptime and creating a new variable with this groupby
team_ranking = full_df.groupby(["RoundYear", "Team"])["LapTime"].median().reset_index(name="MedianLapTime")

# Creating a rank by grouping by each teams round numbers median lap time and ranking
team_ranking["Rank"] = (team_ranking.groupby(["RoundYear"])["MedianLapTime"].rank(method="dense",ascending=True))

In [ ]:
# Merge the rankingh data to main dataframe
full_df = full_df.merge(team_ranking, on=['RoundYear', 'Team'], how="left")